In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook as tqdm

In [ ]:
import logging

logging.basicConfig(format="%(asctime)s [%(process)d] %(levelname)-8s "
                    "%(name)s,%(lineno)s\t%(message)s")
logging.getLogger().setLevel('INFO')

In [ ]:
# Read information to connect to the database and put it in environment variables
import os
with open('ENVVARS.txt') as f:
    for line in f:
        parts = line.split('=')
        if len(parts) == 2:
            os.environ[parts[0]] = parts[1].strip()
            
os.environ['lexicon_name'] = os.environ['dbname']

In [ ]:
db_name = 'ticclat_test'
os.environ['dbname'] = db_name

In [ ]:
from ticclat.ticclat_schema import Lexicon, Wordform, Anahash

from ticclat.dbutils import get_session, session_scope

Session = get_session(os.environ['user'], os.environ['password'], os.environ['dbname'])

In [ ]:
# get variants of input word
from sqlalchemy import select, text
from sqlalchemy.sql import func, distinct, and_, desc

from ticclat.ticclat_schema import MorphologicalParadigm

word = 'wenkbrauw'

with session_scope(Session) as session:
    q = select([Wordform.wordform_id, MorphologicalParadigm.Z, MorphologicalParadigm.Y, MorphologicalParadigm.X, MorphologicalParadigm.W]).select_from(MorphologicalParadigm.__table__.join(Wordform)).where(Wordform.wordform == word)
    r = session.execute(q).fetchone()
    print(r)
    #wf_id = r[0]
    q = select([Wordform, 
                MorphologicalParadigm]).select_from(MorphologicalParadigm.__table__.join(Wordform)).where(and_(MorphologicalParadigm.Z == r.Z,
                                                                                                  MorphologicalParadigm.Y == r.Y,
                                                                                                  MorphologicalParadigm.X == r.X,
                                                                                                  MorphologicalParadigm.W == r.W))
    r = session.execute(q).fetchall()
    for row in r:
        print(row)

In [ ]:
# get time spans of corpora
from ticclat.ticclat_schema import MorphologicalParadigm, corpusId_x_documentId, Corpus, Document

with session_scope(Session) as session:
    q = select([Corpus.name]).select_from(corpusId_x_documentId.join(Document).join(Corpus))
    r = session.execute(q).fetchall()
    for row in r:
        print(row)